# P5


In [163]:
import pandas as pd
import numpy as np

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore')

In [2]:

docs = pd.read_csv("QueryResults.csv")

In [3]:
docs.set_index('Id').sort_index()

,Title,Body,Tags,Score,ViewCount,FavoriteCount,AnswerCount
Id,,,,,,,
4,How to convert a Decimal to a Double in C#?,<p>I want to use a <code>Track-Bar</code> to c...,<c#><floating-point><type-conversion><double><...,759,64063,58,12
11,Calculate relative time in C#,<p>Given a specific <code>DateTime</code> valu...,<c#><datetime><time><datediff><relative-time-s...,1612,188799,552,41
13,Determine a user's timezone,<p>Is there a standard way for a web server to...,<html><browser><timezone><user-agent><timezone...,673,248432,157,27
19,What is the fastest way to get the value of π?,<p>I'm looking for the fastest way to obtain t...,<performance><algorithm><language-agnostic><un...,341,63943,86,23
88,Is gettimeofday() guaranteed to be of microsec...,"<p>I am porting a game, that was originally wr...",<linux><winapi><visual-c++><unix><timer>,104,43767,19,10
...,...,...,...,...,...,...,...
69729326,Endless sine generation in C,<p>I am working on a project which incorporate...,<c><performance><time><precision><trigonometry>,91,10074,25,12
69832748,"Error ""Error: A <Route> is only ever to be use...",<p>I am trying to use routing for the first ti...,<javascript><node.js><reactjs><frameworks><rea...,57,77406,13,14
69875125,find_element_by_* commands are deprecated in s...,<p>When starting the function</p>\n<pre><code>...,<python><selenium><selenium-webdriver><webdriv...,52,59346,25,3


In [4]:
docs.drop(['Id', 'Score', 'ViewCount', 'FavoriteCount','AnswerCount'], axis=1, inplace=True)

## Data Preparation


### Tags preprocessing

In [5]:
# tokenize the tags
def tokenizeTags(tags):
    """
    Remove some unwanted characters
    """
    tags = tags.replace('<', ' ')
    tags = tags.replace('>', '')
    return tags.strip().split(' ')

docs['Tags'] = docs['Tags'].apply(tokenizeTags)

In [6]:
# one doc tags sample
docs['Tags'][0]

['javascript', 'geolocation', 'projection', 'processing.js', 'proj4js']

In [7]:
# get 200 most frequent tags
def getMostFrequentTags(df: pd.DataFrame, col: str, max_tags_count = 200) -> pd.DataFrame:
    all_tags = [item for sublist in df[col] for item in sublist]
    unique, counts = np.unique(all_tags, return_counts=True)
    return pd.DataFrame({'word': unique, 'count':counts}).sort_values('count', ascending=False)[:max_tags_count]

common_tags = getMostFrequentTags(docs, 'Tags').word.values

In [8]:
# filter tags of each document to restrict to most common
def get_most_tag(row):
    return [tag for tag in row if tag in common_tags]

docs['tags'] = docs['Tags'].apply(get_most_tag)

In [9]:
docs.drop('Tags', axis=1, inplace=True)

### Word Cloud

In [162]:
# TODO WIP
# from wordcloud import WordCloud
#
# words = str([tag for row in docs.tags for tag in row])
# # Create a WordCloud object
# wordcloud = WordCloud(background_color="white", max_words=200, contour_width=3, contour_color='steelblue')
# # Generate a word cloud
# wordcloud.generate(words)
# # Visualize the word cloud
# wordcloud.to_image()

### Words preprocessing

In [11]:
# add title into the body
docs['body'] = docs['Title'] + '\n' + docs['Body']

#### Remove code snippets

In [12]:
from bs4 import BeautifulSoup

def remove_code_snippets(docs):
    """
    Remove code snippets from docs
    surrounded by <pre> tags
    """
    # ensure '<' & '>' chars are set, not the unicode char
    docs = docs.apply(lambda d: d.replace('&lt;', '<').replace('&gt;', '>'))
    ret = []
    for doc in docs:
        soup = BeautifulSoup(doc, 'html.parser')
        removals = soup.find_all('pre')
        for pre in removals:
            pre.decompose()
        ret.append(soup.get_text())
    return ret


In [13]:
docs.body = remove_code_snippets(docs.body)

#### Remove Stop words, Lemmatize, Stemmize, Tokenize

In [14]:
# remove stop words
# lemmatize (convert and remove prefixes and suffixes to ignore the gender, plural, verb : walking, walks, walked, walk -> walk)
# and/or stemmize (take the same root of words : universal / university -> univers)

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import EnglishStemmer
from nltk.stem import WordNetLemmatizer

stopwords = set(stopwords.words('english'))
filtre_stopw =  lambda text: [token.lower() for token in text if token.lower() not in stopwords]

# replace words with stem
def tokenize_sw_stem(doc):
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    stemmer = EnglishStemmer()
    filtered = filtre_stopw(tokenizer.tokenize(doc))
    ret = " ".join([stemmer.stem(s) for s in filtered] )
    return ret

# replace words with lem
def tokenize_sw_lem(doc):
    lemmatizer = WordNetLemmatizer()
    filtered = filtre_stopw(nltk.tokenize.word_tokenize(doc))
    # filtered = filtre_stopw(tokenizer.tokenize(doc))
    ret = " ".join([lemmatizer.lemmatize(s) for s in filtered] )
    # TODO manage .net token
    # manage c# token
    ret = re.sub(r'c #', 'c#', ret)
    # temp
    # ret = re.sub(r' #', '', ret)
    # ret = re.sub(r'# ', '', ret)

    # removes punctuation, comes from string.punctuation, without some special characters present into the tags (c#, c++, objective-c, )
    ret = ret.translate(str.maketrans('', '', r"""!"$%&'()*,/:;<=>?@[\]^_`{|}~"""))
    # removes all non-word
    ret = re.sub(' \W+ ',' ', ret)

    # remove single char, except 'c', 'r'
    # ret = re.sub(r'\s[abd-qs-z]{1}\s', ' ', ret)
    ret = re.sub(r'\s[abd-q]{1}\s', ' ', ret)
    ret = re.sub(r'\s[s-z]{1}\s', ' ', ret)
    ret = re.sub(r'\s\+{1}', ' ', ret)

    # ret = re.sub(r'\s[abd-qs-z]{1}\s', ' ', ret)
    # ret = re.sub(r'\s[s-z]{1}', ' ', ret)
    # remove double or more spaces
    ret = re.sub(r' {2,}', ' ', ret)
    return ret


# words_stem = list(map(tokenize_sw_stem, docs.body))
# words_lem = list(map(tokenize_sw_lem, docs.body))

# test

test_string = "this is a test string. Contains with __ chars and c++, C# tags, java, .net, r language, c lang , ; and some isolated letters a b c d e z x end signs + "
tokenize_sw_lem(test_string)

print('C#')
index = 3632
docs.body[index]
tokenize_sw_lem(docs.body[index])
print('c++')
index = 70
docs.body[index]
tokenize_sw_lem(docs.body[index])
print('.net')
index = 1460
docs.body[index]
tokenize_sw_lem(docs.body[index])

print('MISC')
index = 203
docs.body[index]
tokenize_sw_lem(docs.body[index])

'test string contains char c++ c# tag java .net r language c lang isolated letter c x end sign '

C#


"c# - WebRequest HTTP POST with Cookie (port from curl script)\nThe IBM RTC RESTful api gives an example of a shell script for authenticating with the server:\n\nThis works perfectly, however i need to authenticate with the server using c#.\nSo far i have the following, but it isn't working (returns the authorization failed page):\n\n"

'c# webrequest http post cookie port curl script ibm rtc restful api give example shell script authenticating server work perfectly however need authenticate server using c# far following nt working return authorization failed page '

c++


...

".NET Web API 2 OWIN Bearer Token Authentication\nI'm implementing a Web API 2 service architecture in my .NET web application.  The client consuming the requests is pure javascript, no mvc/asp.net.  I'm using OWIN to try to enable token authentication per this article OWIN Bearer Token Authentication with Web API Sample.  I seem to be missing something with the authentication step after its authorized.\nMy login looks like:\n\nIt returns\n\nThen I try to set the HTTP header Bearer on further requests in AngularJS like:\n\nto a API like:\n\nbut no matter what I do this service is 'unauthorized'.  Am I missing something here?\n"

'.net web api 2 owin bearer token authentication implementing web api 2 service architecture .net web application client consuming request pure javascript mvcasp.net using owin try enable token authentication per article owin bearer token authentication web api sample seem missing something authentication step authorized login look like return try set http header bearer request angularjs like api like matter service unauthorized missing something '

MISC


"Hosting the .NET runtime in a Delphi Program\nI was looking into using some .NET code from within a Delphi program, I will need to make my program extensible using .net assemblies and predefined functions (I already support regular DLLs).\nAfter a lot of searching online, I found  Managed-VCL, but I'm not ready to pay $250 for what I need, I also found some newsgroups with code that's incomplete and doesn't work.\nI'm using Delphi 2007 for win32. What can I use to dynamically execute a function from an assembly with predefined parameters?\nSomething like:\n\nI just want to add that I need to be able to load an arbitrary assemblies (maybe all the assemblies in a specific folder), so creating a C# wrapper may not work.\n"

'hosting .net runtime delphi program looking using .net code within delphi program need make program extensible using .net assembly predefined function already support regular dlls lot searching online found managed-vcl ready pay 250 need also found newsgroups code incomplete nt work using delphi 2007 win32 use dynamically execute function assembly predefined parameter something like want add need able load arbitrary assembly maybe assembly specific folder creating c# wrapper may work .'

In [15]:
# let's choose the lemmatize version of the corpus, since it results in real words that could be used as tags
docs['tokenized_body'] = list(map(tokenize_sw_lem, docs.body))

#### Remove remaining 'bad words'


In [16]:
import re

def remove_words_old(words):
    ret = []
    for word in words:
        # print(f'\nprocessing sentence: {word}\n')
        flat = re.sub(r'[0-9]+', '', word) # words that are numbers
        flat = re.sub(r'_+', '', flat) # words that contain underscores
        flat = re.sub(r'-+', '', flat) # words that contain dashes
        flat = re.sub(r'\+', '', flat)
        flat = re.sub(r'\w{15,}', '', flat) # words longer than 15 chars are most probably code (i.e. myClass.myMethodName())
        flat = " ".join(flat.split()) # remove double spaces
        # print(f'\ncurrated : {flat}')
        ret.append(flat)
    return ret

def remove_words(words):
    ret = []
    # print(f'\nprocessing sentence: {word}\n')
    flat = re.sub(r'[0-9]+', '', words) # words that are numbers
    flat = re.sub(r'_+', '', flat) # words that contain underscores
    flat = re.sub(r'-+', '', flat) # words that contain dashes
    # flat = re.sub(r'\+', '', flat)
    flat = re.sub(r'\w{15,}', '', flat) # words longer than 15 chars are most probably code (i.e. myClass.myMethodName())
    flat = " ".join(flat.split()) # remove double spaces
    # print(f'\ncurrated : {flat}')
    ret.append(flat)
    return flat

print('MISC')
index = 203
docs.body[index]
remove_words(docs.body[index])

MISC


"Hosting the .NET runtime in a Delphi Program\nI was looking into using some .NET code from within a Delphi program, I will need to make my program extensible using .net assemblies and predefined functions (I already support regular DLLs).\nAfter a lot of searching online, I found  Managed-VCL, but I'm not ready to pay $250 for what I need, I also found some newsgroups with code that's incomplete and doesn't work.\nI'm using Delphi 2007 for win32. What can I use to dynamically execute a function from an assembly with predefined parameters?\nSomething like:\n\nI just want to add that I need to be able to load an arbitrary assemblies (maybe all the assemblies in a specific folder), so creating a C# wrapper may not work.\n"

"Hosting the .NET runtime in a Delphi Program I was looking into using some .NET code from within a Delphi program, I will need to make my program extensible using .net assemblies and predefined functions (I already support regular DLLs). After a lot of searching online, I found ManagedVCL, but I'm not ready to pay $ for what I need, I also found some newsgroups with code that's incomplete and doesn't work. I'm using Delphi for win. What can I use to dynamically execute a function from an assembly with predefined parameters? Something like: I just want to add that I need to be able to load an arbitrary assemblies (maybe all the assemblies in a specific folder), so creating a C# wrapper may not work."

In [17]:
docs['tokenized_body_cleaned'] = docs.tokenized_body.apply(remove_words)


In [18]:
index = 28
docs.body[index]
docs.tokenized_body[index]
docs.tokenized_body_cleaned[index]

'How to mock window.location.href with Jest + Vuejs?\nCurrently, I am implementing unit test for my project and there is a file that contained window.location.href.\nI want to mock this to test and here is my sample code:\n\nBut I get this error:\n\nI had tried some solutions but not resolve it. I need some hints to help me get out of this trouble. Plz help.\n'

'mock window.location.href jest vuejs currently implementing unit test project file contained window.location.href want mock test sample code get error tried solution resolve need hint help get trouble plz help .'

'mock window.location.href jest vuejs currently implementing unit test project file contained window.location.href want mock test sample code get error tried solution resolve need hint help get trouble plz help .'

In [19]:
docs[docs.body.str.contains('...')]

,Title,Body,tags,body,tokenized_body,tokenized_body_cleaned
0,Mercator longitude and latitude calculations t...,<p>I have this image. It's a map of the UK (no...,[javascript],Mercator longitude and latitude calculations t...,mercator longitude latitude calculation croppe...,mercator longitude latitude calculation croppe...
1,Starting point for learning CAD/CAE file formats?,<p>We are developing some stress and strain an...,[c++],Starting point for learning CAD/CAE file forma...,starting point learning cadcae file format dev...,starting point learning cadcae file format dev...
2,LINQ query on a DataTable,<p>I'm trying to perform a LINQ query on a Dat...,"[c#, .net, linq]",LINQ query on a DataTable\nI'm trying to perfo...,linq query datatable trying perform linq query...,linq query datatable trying perform linq query...
3,How to overload std::swap(),<p><code>std::swap()</code> is used by many st...,"[c++, performance, optimization, stl]",How to overload std::swap()\nstd::swap() is us...,overload std swap std swap used many std conta...,overload std swap std swap used many std conta...
4,How do you get a directory listing in C?,<p>How do you scan a directory for folders and...,"[c, file]",How do you get a directory listing in C?\nHow ...,get directory listing c scan directory folder ...,get directory listing c scan directory folder ...
...,...,...,...,...,...,...
27123,Can't perform a React state update on an unmou...,<h2>Problem</h2>\n\n<p>I am writing an applica...,"[javascript, reactjs, typescript]",Can't perform a React state update on an unmou...,ca nt perform react state update unmounted com...,ca nt perform react state update unmounted com...
27124,Can WPF applications be run in Linux or Mac wi...,<p>Microsoft announced .NET Core 3 comes with ...,"[linux, wpf]",Can WPF applications be run in Linux or Mac wi...,wpf application run linux mac .net core 3 micr...,wpf application run linux mac .net core micros...
27125,Instance of 'SQLAlchemy' has no 'Column' membe...,<p>I'm currently trying to implement steam log...,"[python, python-3.x]",Instance of 'SQLAlchemy' has no 'Column' membe...,instance sqlalchemy column member no-member cu...,instance sqlalchemy column member nomember cur...
27126,"Why does `if None.__eq__(""a"")` seem to evaluat...",<p>If you execute the following statement in P...,"[python, python-3.x, string]","Why does `if None.__eq__(""a"")` seem to evaluat...",none.eq seem evaluate true quite execute foll...,none.eq seem evaluate true quite execute follo...


## Corpus size and Vocabulary

In [20]:
# count the total number of words in the overall corpus
# ideally would to be limited to 2000 / 3000 words maximum

def get_common_vocabulary():
    lem = map(lambda x : x.split(), docs.tokenized_body_cleaned)
    flat_list = [word for sublist in list(lem) for word in sublist ]
    # len(flat_list)
    unique, counts = np.unique(flat_list, return_counts=True)
    counted_words =  pd.DataFrame({'word': unique, 'count': counts}).sort_index(ascending=False)
    min_counts = 1000
    # take the 3000 most used words in the corpus for our vocabulary
    vocabulary = pd.DataFrame({'count': counts, 'word': unique}).sort_values('count', ascending=False)[:min_counts].word.values
    # len(vocabulary)
    # add to vocabulary the list of common tags
    return np.unique(np.append(vocabulary, common_tags))

common_words = get_common_vocabulary()
common_words = ' '.join(common_words)
common_words = re.sub(r'\.{3}', '', str(common_words))
common_words = re.sub(r'\.{2}', '', common_words)
common_words = re.sub(r'\. ', '', common_words)
common_words = re.sub(r'[ ]{2,}', '', common_words)
common_words = np.array(common_words.split(' '))
common_words

array(['.net', 'able', 'access', ..., 'year', 'yes', 'yet'], dtype='<U22')

In [21]:
# reduce our corpus to the common vocabulary

def reduce_body(body):
    ret = []
    for word in body.split(' '):
        # if word in np.append(common_words, common_tags):
        if word in common_words:
            ret.append(word)
    return ' '.join(ret)

# test
# reduce_body(docs.tokenized_body[1])

docs['reduced_body'] = docs.tokenized_body_cleaned.apply(reduce_body)

## MultiLabel Binarizer

### Tags

In [22]:
from sklearn.preprocessing import MultiLabelBinarizer

def mlb_tags():
    mlb_t = MultiLabelBinarizer()
    # fit the mlb with common tags (200)
    mlb_t.fit([common_tags])
    print(mlb_t.classes_.shape)
    docs['mlb_tags'] = mlb_t.transform(docs.tags).tolist()
    return mlb_t

def mlb_body():
    mlb_b = MultiLabelBinarizer()
    mlb_b.fit([common_words])
    print(mlb_b.classes_.shape)
    docs['mlb_reduced_body'] = mlb_b.transform(docs.reduced_body).tolist()
    return mlb_b


In [23]:
mlb_tags()

(200,)


MultiLabelBinarizer()

In [24]:
mlb_b = mlb_body()

(1102,)


In [25]:
# test mlb on body
# c = mlb_b.classes_
# np.sort(docs.reduced_body[0].split())
test_str = 'java'

# df = pd.DataFrame(mlb_b.transform([[test_str]]).T)
# index = df[df[0] == 1].index.values[0]
# assert mlb_b.classes_[index] == test_str
mlb_b.inverse_transform(mlb_b.transform([[test_str]]))

[('java',)]

## Train, Test Sets

In [26]:
from sklearn.model_selection import train_test_split

# X = docs.reduced_body
# y = docs.mlb_tags

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## Bag Of Words / Tf-Idf 

Bag of words uses CountVectorizer, then we look for the most frequent word to suggest a tag

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(vocabulary=common_words)
bow = cv.fit_transform(docs.reduced_body)
data_dense = bow.todense()
print("Sparsity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")


Sparsicity:  2.8385362449229063 %


In [28]:

bow.toarray()
cv.get_feature_names_out()

array([[0, 2, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

array(['.net', 'able', 'access', ..., 'year', 'yes', 'yet'], dtype=object)

Tf-idf takes the frequency of a word from a sentence in a context of a corpus, then we take the most frequent words to suggest a tag

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(vocabulary=common_words, ngram_range=(1, 2))
vectorizer.fit(docs['reduced_body'])
vectorizer.get_feature_names_out()


TfidfVectorizer(ngram_range=(1, 2),
                vocabulary=array(['.net', 'able', 'access', ..., 'year', 'yes', 'yet'], dtype='<U22'))

array(['.net', 'able', 'access', ..., 'year', 'yes', 'yet'], dtype=object)

In [30]:
with open('vectorizer', 'wb') as v :
    pickle.dump(vectorizer, v)


#### Prediction method for tf-idf

In [31]:
# predicition - nouvelle methode
# TODO optimize  - temps de calcul trop long ~ 3'30

def predict_tfidf(doc, filter_tags=True) -> list[str]:
    """
    Returns a list of tags representative to the document provided, predicted from tf-idf

    Parameters
    ----------
    doc: iterable
        a list of words to compute tf-idf
    
    filter_tags: boolean
        defines if returned tags must be part of a currated list of tags
    
    Returns
    -------
    ret : a list of strings, may be empty

    """
    res = vectorizer.transform([doc]).T.todense()
    res = pd.DataFrame({'tfidf': res.tolist()}).sort_values('tfidf', ascending=False)[:5]
    res = [ vectorizer.get_feature_names_out()[found_tag_index] for found_tag_index in res.index]    
    # print(f'found tags from tfidf: {res}')
    if (not filter_tags):
        return res
    else:
        return [tag for tag in res if tag in common_tags.word.values]


# test
index = 2
docs.reduced_body[index]
predict_tfidf(docs.reduced_body[index], False) #show all tags found
predict_tfidf(docs.reduced_body[index]) # show only tags found which are part of currated tags (list of 200)


'linq query trying perform linq query object finding query example allowed get something like working linq query allowed'

['linq', 'query', 'allowed', 'finding', 'perform']

AttributeError: 'numpy.ndarray' object has no attribute 'word'

In [ ]:

docs['predicted_tags_tfidf'] = [predict_tfidf(doc) for doc in docs.reduced_body]


In [ ]:
# compute mlb of predicted tags
# docs['mlb_tags_pred_tfidf'] = [mlb.transform([tags]) for tags in docs.predicted_tags_tfidf]
docs['mlb_tags_pred_tfidf'] = mlb_b.transform(docs.predicted_tags_tfidf).tolist()


In [ ]:
len(mlb_b.classes_.tolist())
# docs.mlb_tags_pred_tfidf[0]

#### Multi-label scoring

<https://medium.datadriveninvestor.com/predicting-tags-for-the-questions-in-stack-overflow-29438367261e>

<https://towardsdatascience.com/evaluating-multi-label-classifiers-a31be83da6ea>



In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

y_true = mlb_b.transform(docs.tags).tolist()
y_pred = mlb_b.transform(docs.predicted_tags_tfidf).tolist()

accuracy_score(y_true, y_pred)
average_precision_score(y_true, y_pred)
# recall_score(y_true, y_pred, average='macro')
print(classification_report(y_true, y_pred, target_names=mlb_b.classes_))

### Conclusion

Using tf-idf is not really performant to predict relevant tags (F1 score is way lower than 1)

| Score | value|
|--------|-------|
| Precision score | 0.41|
| Recall score | 0.23|
| f1-score | 0.26|


average parameter is required for multiclass/multilabel targets.

```
'None' : the scores for each class are returned. Otherwise, this determines the type of averaging performed on the data:

'micro': Calculate metrics globally by counting the total true positives, false negatives and false positives.

'macro': Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

'weighted': Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.
```

Plus le recall est haut, plus le modèle repère de positif.

Plus la precision est haute, moins le modèle se trompe sur les positifs

Le F1 Score permet d’effectuer une bonne évaluation de la performance de notre modèle

reminder : <https://inside-machinelearning.com/recall-precision-f1-score/>

## LDA

<https://towardsdatascience.com/latent-dirichlet-allocation-for-topic-modelling-explained-algorithm-and-python-scikit-learn-c65a82e7304d>

In [196]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=1000, random_state=0, n_jobs=-1)
lda.fit(bow)

LatentDirichletAllocation(n_components=1000, n_jobs=-1, random_state=0)

In [197]:
for idx, topic in enumerate(lda.components_):
    print ("Topic ", idx, " ".join(cv.get_feature_names()[i] for i in topic.argsort()[:-3 -1:-1]))

Topic  0 statement using nt
Topic  1 div instead like
Topic  2 target error setting
Topic  3 almost nt answer
Topic  4 code including saying
Topic  5 opencv using code
Topic  6 task run way
Topic  7 browser firefox work
Topic  8 numpy like python
Topic  9 bean spring created
Topic  10 php use using
Topic  11 happens fix problem
Topic  12 lock code using
Topic  13 whether changed really
Topic  14 read reading file
Topic  15 obvious nt seems
Topic  16 project option ve
Topic  17 remove want trying
Topic  18 avoid like using
Topic  19 vector way want
Topic  20 way wrong using
Topic  21 result get term
Topic  22 actual really simple
Topic  23 people program question
Topic  24 le using problem
Topic  25 project repository feature
Topic  26 record disk sqlite
Topic  27 appreciated would help
Topic  28 may solution need
Topic  29 rule installation able
Topic  30 prevent since still
Topic  31 buffer data using
Topic  32 great would work
Topic  33 design let like
Topic  34 logic code use
Topic 

In [198]:
def predict_lda(text):
    best_topic_indexes = lda.transform(cv.transform([text])).argsort()[0][::-1][0]
    print(f'lda best topic index found: {best_topic_indexes}')
    best_topics = lda.components_[best_topic_indexes].argsort()[::-1]
    print(f'topic\'s word indexes {best_topics[:1]}')
    ret = [cv.get_feature_names()[topic] for topic in best_topics[:1]]
    # print(f'predicted tag: {ret}')
    return ret

In [211]:
# some indexes that work, otherwise most of the time predicted tag is wrong
# index = 7894
index = 1250

text = docs.reduced_body[index]
print(f'input text: {text}')
print(f'known tags: {docs.tags[index]}')

predict_lda(text)


input text: bit x machine code like first byte x disk file using suggestion linux file
known tags: ['linux', 'assembly', 'x86']
lda best topic index found: 776
topic's word indexes [552]


['linux']

## Word2Vect

In [41]:
# snippet taken from sample notebook
import gensim
import multiprocessing
from gensim.models import Word2Vec

sentences = [ docs.split() for docs in docs.reduced_body]

print("Build & train Word2Vec model ...")
w2v_model = Word2Vec(sentences, min_count=1,vector_size=200, workers=multiprocessing.cpu_count())
# w2v_model.build_vocab(sentences)
# w2v_model.train(sentences, total_examples=1, epochs=1)
model_vectors = w2v_model.wv
w2v_words = model_vectors.index_to_key
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

Build & train Word2Vec model ...
Vocabulary size: 3037
Word2Vec trained


In [42]:
# training another model, will give slightly different results

model = Word2Vec(sentences, min_count=1, vector_size=200, workers=multiprocessing.cpu_count())
# model.build_vocab(sentences)
# model.train(sentences, total_examples=1, epochs=1)
# model.wv.index_to_key
print("Vocabulary size: %i" % len(model.wv.index_to_key))

Vocabulary size: 3037


In [43]:
# guess close words
len(model.wv.index_to_key)
guess = 'java'
model.wv[guess]
model.wv.most_similar(guess)
w2v_model.wv[guess]
w2v_model.wv.most_similar(guess)

3037

array([-0.58557546, -1.1354102 ,  0.12046065,  0.59639007, -0.45297256,
       -0.08538836, -1.0288872 , -0.07502311,  0.34875378,  1.456882  ,
        0.02904   ,  0.3484971 ,  0.8561433 ,  0.44659573,  0.21644755,
       -0.67796785, -0.28746688, -0.79099214, -0.89820534,  0.33728933,
        0.01065614,  0.05802602,  0.52821654, -0.4307533 ,  1.4652115 ,
        0.40524253,  0.2742625 , -1.0524437 , -0.22509259,  0.6129267 ,
        0.7393892 ,  1.0600578 ,  0.03563976,  0.39282697, -1.088671  ,
        0.18768804, -0.88462496,  0.34919947, -0.39016986, -1.9281803 ,
       -0.7560163 , -0.4093855 , -0.12985916,  0.11784337,  0.2488027 ,
       -0.5095998 ,  0.34837845, -0.620043  ,  0.22599325,  0.8957733 ,
        1.5080509 , -1.3247198 ,  0.94050956,  0.48689887, -0.6084543 ,
       -0.68195045, -0.246007  ,  0.7372067 , -0.51015025, -0.08187965,
       -0.33196554, -1.0757642 , -1.3153936 , -0.5359754 ,  0.9020954 ,
        0.32251906,  0.78391486,  0.93809205,  0.74706566, -0.82

[('ee', 0.7697761058807373),
 ('scala', 0.6680197715759277),
 ('swing', 0.5824506878852844),
 ('clojure', 0.5337738990783691),
 ('kotlin', 0.5332742929458618),
 ('jdk', 0.5242394208908081),
 ('jboss', 0.5157983303070068),
 ('c#', 0.5032386183738708),
 ('cc++', 0.5027136206626892),
 ('jni', 0.4903098940849304)]

array([-0.52990884, -1.0066278 ,  0.25581354,  1.3836367 , -0.49463156,
       -0.21418215, -0.27241096,  0.77627134,  0.6285232 ,  1.3047533 ,
        0.02917864,  0.536986  ,  0.2684114 , -0.25338364, -0.40457472,
       -0.21228805, -0.4150675 , -0.66024923, -0.16639598,  0.3857961 ,
       -0.2543728 ,  0.14833833,  0.39891765, -0.79374015,  1.5856831 ,
        0.23168905, -0.25601336, -0.8900388 ,  0.6595844 ,  0.4509691 ,
        0.5156059 ,  0.7576896 ,  0.47992414, -0.06808856, -0.77450997,
       -0.29349303, -1.1878793 , -0.32424703, -0.02394585, -2.115284  ,
        0.00647559, -0.5236665 , -0.18572758,  0.33532807, -0.7628372 ,
       -1.0393735 ,  0.13922833, -0.13380443,  0.11398205,  0.8548549 ,
        1.739255  , -1.1436307 ,  1.0189074 ,  0.23138583, -0.9459974 ,
       -1.0086391 ,  0.28035676,  0.8386735 , -0.11423967,  0.07597468,
       -0.38157305, -0.5324276 , -1.0570855 , -0.31051698,  0.55488914,
        0.2617658 ,  0.41912267,  0.21883406,  0.3338311 , -0.35

[('ee', 0.8012250661849976),
 ('scala', 0.6843573451042175),
 ('swing', 0.6296717524528503),
 ('kotlin', 0.5472506284713745),
 ('cc++', 0.5280548334121704),
 ('clojure', 0.5243746042251587),
 ('c#', 0.5197808742523193),
 ('jvm', 0.5111938714981079),
 ('jni', 0.4784856140613556),
 ('pure', 0.4744231402873993)]

In [44]:
with open('w2v_model', 'wb') as v :
    # pickle.dump(w2v_model, v)
    w2v_model.save(v)


### Averaged vector for documents

In [45]:
# compute average vector for each document
averaged_docs = [np.mean([w2v_model.wv[word] for word in doc.split()], axis=0) for doc in docs.reduced_body ]
# averaged_docs
docs['wv_averaged_doc'] = averaged_docs

In [46]:
#

### Clusterize embeded documents

*** last update ***

In [ ]:
from sklearn.cluster import KMeans


X = averaged_docs
kmeans = KMeans(n_clusters=200, random_state=0).fit(X)

kmeans.labels_

# kmeans.predict([[0, 0], [12, 3]])

# kmeans.cluster_centers_


In [49]:
len(averaged_docs[0])


200

In [ ]:
# TODO classify 
# calculer le doc2vect ? --> le vecteur pour un document, pas pour juste les mots
# faire une reduction de dimension PCA
# faire un KMeans pour trouver des clusters


## Appendix

In [ ]:
# vectorize a whole doc
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features
    
vectorized_docs = vectorize(docs.reduced_body, model=w2v_model)
len(vectorized_docs), len(vectorized_docs[0])

(27128, 200)

In [ ]:
vectorized_docs

27128

In [ ]:
import sklearn.metrics as metrics

#  from https://www.kaggle.com/code/michaelfumery/stackoverflow-questions-tag-generator/notebook?scriptVersionId=68023262

def metrics_score(model, df, y_true, y_pred):
    """Compilation function of metrics specific to multi-label
    classification problems in a Pandas DataFrame.
    This dataFrame will have 1 row per metric
    and 1 column per model tested. 

    Parameters
    ----------------------------------------
    model : string
        Name of the tested model
    df : DataFrame 
        DataFrame to extend. 
        If None : Create DataFrame.
    y_true : array
        Array of true values to test
    y_pred : array
        Array of predicted values to test
    ----------------------------------------
    """
    if(df is not None):
        temp_df = df
    else:
        temp_df = pd.DataFrame(index=["Accuracy", "F1",
                                      "Jaccard", "Recall",
                                      "Precision"],
                               columns=[model])
        
    scores = []
    scores.append(metrics.accuracy_score(y_true, 
                                         y_pred))
    scores.append(metrics.f1_score(y_pred, 
                                   y_true, 
                                   average='weighted'))
    scores.append(metrics.jaccard_score(y_true, 
                                        y_pred, 
                                        average='weighted'))
    scores.append(metrics.recall_score(y_true, 
                                       y_pred, 
                                       average='weighted'))
    scores.append(metrics.precision_score(y_true, 
                                          y_pred, 
                                          average='weighted'))
    temp_df[model] = scores
    
    return temp_df

à lire


topic modelling : unsupervised learning

topic classification : supervised learning


<https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/>

<https://www.baeldung.com/cs/ml-word2vec-topic-modeling>

<https://medium.com/le-blog-de-lapprentissage-automatique/pr%C3%A9diction-des-tags-des-questions-de-stack-overflow-9be00f7672f9>

